In [16]:
import pandas as pd
import yfinance as yf
from datetime import date
import numpy as np
import matplotlib.pyplot as plt

In [17]:
df = pd.read_csv("data/assets july 7 2025.csv")
df.head()

,Asset,Ticker,Units,Purchase Price,Currency Purchase,Currency Yahoo,Price Last Update,Date Last Update,Value Last Update,Profit Last Update
0,S&P 500 ETF,SXR8.DE,17.4780,584.85,EUR,EUR,560.36,2025-06-30,9793.972080,-428.036220
1,Zalando,ZAL.DE,407.0000,44.08,EUR,EUR,28.16,2025-06-30,11461.120000,-6479.440000
2,S&P 500 IT ETF,QDVE.DE,340.7460,30.68,EUR,EUR,30.85,2025-06-30,10512.014100,57.926820
3,Bitcoin,BTC-USD,0.1279,44894.00,EUR,USD,91958.00,2025-06-30,11761.428200,6019.485600
4,Microsoft,MSFT,0.0389,384.06,EUR,USD,422.55,2025-06-30,16.437195,1.497261


In [18]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Asset               5 non-null      str    
 1   Ticker              5 non-null      str    
 2   Units               5 non-null      float64
 3   Purchase Price      5 non-null      float64
 4   Currency Purchase   5 non-null      str    
 5   Currency Yahoo      5 non-null      str    
 6   Price Last Update   5 non-null      float64
 7   Date Last Update    5 non-null      str    
 8   Value Last Update   5 non-null      float64
 9   Profit Last Update  5 non-null      float64
dtypes: float64(5), str(5)
memory usage: 532.0 bytes


# FX Rate

In [19]:
fx_pair = 'USDEUR=X'  # Yahoo Finance symbol for USD to EUR
fx_data = yf.Ticker(fx_pair).history(period="1d")
fx_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2026-02-13 00:00:00+00:00,0.84248,0.84397,0.84179,0.84257,0,0.0,0.0


In [20]:
# Fetch FX rate for USD → EUR
def get_fx_rate(from_currency: str, to_currency: str = 'EUR') -> float:
    """
    Get live FX rate from Yahoo. Returns 1.0 if from/to are the same.
    """
    if from_currency == to_currency:
        return 1.0
    pair = f"{from_currency}{to_currency}=X"
    try:
        data = yf.Ticker(pair).history(period="1d")
        return data["Close"].iloc[-1]
    except Exception as e:
        print(f"Error fetching FX rate: {e}")
        return np.nan

In [21]:
# test the get_fx_rate
print(get_fx_rate('USD', 'EUR'))

0.8425700068473816


In [22]:
# Cache FX rates with EUR = 1.0
fx_cache = {}

for ccy in df['Currency Yahoo'].unique():
    if ccy == 'EUR':
        fx_cache[ccy] = 1.0  # no conversion needed
    else:
        fx_cache[ccy] = get_fx_rate(ccy)
print("FX Cache:", fx_cache)

FX Cache: {'EUR': 1.0, 'USD': np.float64(0.8425700068473816)}


In [23]:
# Download one price from a ticker for a specific date
yf.Ticker('BTC-USD').history(period="1d")['Close']

Date
2026-02-15 00:00:00+00:00    69035.515625
Name: Close, dtype: float64

In [24]:
def get_price_eur(row) -> float:
    """
    Fetch Yahoo price in native currency and convert to EUR.
    """
    try:
        price_native = (
            yf.Ticker(row['Ticker'])
            .history(period="1d")['Close']
            .iloc[-1]
        )
        fx_rate = fx_cache.get(row['Currency Yahoo'], 1.0)  # default 1.0
        return price_native * fx_rate
    except Exception as e:
        print(f"Error fetching {row['Ticker']}: {e}")
        return np.nan

In [25]:
# get the price for all tickers
df.apply(get_price_eur, axis=1)

0      619.359985
1       20.510000
2       33.555000
3    58167.254873
4      338.140201
dtype: float64

In [26]:
# --- CALCULATIONS ---------------------------------------------
today = date.today()

df['Price Today (EUR)'] = df.apply(get_price_eur, axis=1)
df['Value Today (EUR)'] = df['Units'] * df['Price Today (EUR)']

df['Gain € Since Last'] = df['Value Today (EUR)'] - df['Value Last Update']
df['Gain € Since Purchase'] = df['Value Today (EUR)'] - df['Units'] * df['Purchase Price']

df['Gain % Since Last'] = df['Gain € Since Last'] / df['Value Last Update'] * 100
df['Gain % Since Purchase'] = df['Gain € Since Purchase'] / (df['Units'] * df['Purchase Price']) * 100


In [27]:
df.head()

,Asset,Ticker,Units,Purchase Price,Currency Purchase,Currency Yahoo,Price Last Update,Date Last Update,Value Last Update,Profit Last Update,Price Today (EUR),Value Today (EUR),Gain € Since Last,Gain € Since Purchase,Gain % Since Last,Gain % Since Purchase
0,S&P 500 ETF,SXR8.DE,17.4780,584.85,EUR,EUR,560.36,2025-06-30,9793.972080,-428.036220,619.359985,10825.173824,1031.201744,603.165524,10.528943,5.900656
1,Zalando,ZAL.DE,407.0000,44.08,EUR,EUR,28.16,2025-06-30,11461.120000,-6479.440000,20.510000,8347.570093,-3113.549907,-9592.989907,-27.166192,-53.470961
2,S&P 500 IT ETF,QDVE.DE,340.7460,30.68,EUR,EUR,30.85,2025-06-30,10512.014100,57.926820,33.555000,11433.732134,921.718034,979.644854,8.768234,9.370927
3,Bitcoin,BTC-USD,0.1279,44894.00,EUR,USD,91958.00,2025-06-30,11761.428200,6019.485600,58174.627360,7440.534839,-4320.893361,1698.592239,-36.737829,29.582188
4,Microsoft,MSFT,0.0389,384.06,EUR,USD,422.55,2025-06-30,16.437195,1.497261,338.140201,13.153654,-3.283541,-1.786280,-19.976287,-11.956413


In [28]:
# --- Totals Row -----------------------------------------------
totals = {
    'Asset': 'TOTAL',
    'Ticker': '',
    'Gain € Since Last': df['Gain € Since Last'].sum(),
    'Gain % Since Last': df['Gain € Since Last'].sum() / df['Value Last Update'].sum() * 100,
    'Gain € Since Purchase': df['Gain € Since Purchase'].sum(),
    'Gain % Since Purchase': df['Gain € Since Purchase'].sum() / (df['Units'] * df['Purchase Price']).sum() * 100,
}

In [30]:
# Columns to show
report_cols = [
    'Asset', 'Ticker',
    'Gain € Since Last', 'Gain % Since Last',
    'Gain € Since Purchase', 'Gain % Since Purchase'
]

report = pd.concat([df[report_cols], pd.DataFrame([totals])], ignore_index=True)
report

,Asset,Ticker,Gain € Since Last,Gain % Since Last,Gain € Since Purchase,Gain % Since Purchase
0,S&P 500 ETF,SXR8.DE,1031.201744,10.528943,603.165524,5.900656
1,Zalando,ZAL.DE,-3113.549907,-27.166192,-9592.989907,-53.470961
2,S&P 500 IT ETF,QDVE.DE,921.718034,8.768234,979.644854,9.370927
3,Bitcoin,BTC-USD,-4320.893361,-36.737829,1698.592239,29.582188
4,Microsoft,MSFT,-3.283541,-19.976287,-1.786280,-11.956413
5,TOTAL,,-5484.807031,-12.595730,-6313.373570,-14.227789


In [31]:
# --- Print Report ---------------------------------------------
print(f"\nSnapshot as of {today} (all converted to EUR):\n")
print(report.to_string(index=False, float_format='{:,.2f}'.format))


Snapshot as of 2026-02-15 (all converted to EUR):

         Asset  Ticker  Gain € Since Last  Gain % Since Last  Gain € Since Purchase  Gain % Since Purchase
   S&P 500 ETF SXR8.DE           1,031.20              10.53                 603.17                   5.90
       Zalando  ZAL.DE          -3,113.55             -27.17              -9,592.99                 -53.47
S&P 500 IT ETF QDVE.DE             921.72               8.77                 979.64                   9.37
       Bitcoin BTC-USD          -4,320.89             -36.74               1,698.59                  29.58
     Microsoft    MSFT              -3.28             -19.98                  -1.79                 -11.96
         TOTAL                  -5,484.81             -12.60              -6,313.37                 -14.23
